In [1]:
print("Hello, World!")

Hello, World!


In [2]:
from pydantic import BaseModel

# Pydantic

In [3]:
class Account(BaseModel):
    name: str
    address: str
    group_id: str
    balance: float

In [4]:
class AccountJoin(BaseModel):
    name: str
    address: str
    group_id: str

In [5]:
class FoodInsert(BaseModel):
    name: str
    insert_time: str
    due_date: str | None = None
    money_cost: float
    buyer_address: str
    group_id: str
    notes: str | None = None

In [6]:
class Food(BaseModel):
    name: str
    insert_time: str
    due_date: str | None = None
    money_cost: float
    buyer_address: str
    quantity_left_percentage: float
    food_address: str
    simpler_food_address: str
    group_id: str
    notes: str | None = None

In [7]:
class FoodPie(BaseModel):
    name: str
    rate: float

In [8]:
class FoodUsageRecord(BaseModel):
    use_time: str
    use_food_group_id: str
    use_food_address: str
    use_food_amount_percentage: float
    user_address: str
    notes: str

In [9]:
class TransferRequest(BaseModel):
    transfer_time: str
    from_address: str
    to_address: str
    amount: float
    group_id: str
    notes: str | None = None

In [10]:
class TransferRecord(BaseModel):
    transfer_time: str
    from_address: str
    from_name: str
    to_address: str
    to_name: str
    amount: float
    group_id: str
    notes: str | None = None

# Database

In [11]:
import random
def random_address():
    hex_chars = '0123456789abcdef'
    random_hex = ''.join(random.choice(hex_chars) for _ in range(40))
    return f"0x{random_hex}"

In [12]:
class Database:
    def __init__(self):
        self.foods: dict[str, list[Food]] = dict()
        self.groups: dict[str, list[Account]] = dict()
        self.transactions: dict[str, list[TransferRecord]] = dict()
        
    def insert_food(self, fi: FoodInsert):
        address = random_address()
        f = Food(
            name = fi.name,
            insert_time = fi.insert_time,
            due_date = fi.due_date,
            money_cost = fi.money_cost,
            buyer_address = fi.buyer_address,
            quantity_left_percentage = 100,
            food_address = address,
            simpler_food_address = address[:4],
            group_id = fi.group_id,
            notes = fi.notes
        )
        if fi.group_id not in self.foods:
            self.foods[fi.group_id] = list()
        self.foods[fi.group_id].append(f)
        return {
            "message": "成功新增食物"
        }

    def single_food(self, address: str, group_id: str):
        
        the_food_group = list()

        if group_id in self.foods:
            the_food_group = self.foods[group_id]
            
        for f in the_food_group:
            if f.food_address == address:
                return f
        return None

# class Food(BaseModel):
#     name: str
#     insert_time: str
#     due_date: str | None = None
#     money_cost: float
#     buyer_address: str
#     quantity_left_percentage: float
#     food_address: str
#     simpler_food_address: str
#     group_id: str
#     notes: str | None = None
    
    def all_foods(self, group_id):
        if group_id not in self.foods:
            empty_list = list()
            empty_list.append(
                Food(
                    name="目前尚無庫存",
                    insert_time="入庫日期",
                    due_date="保存期限",
                    money_cost=0,
                    buyer_address="",
                    quantity_left_percentage=0.0,
                    food_address="食材地址",
                    simpler_food_address="",
                    group_id=group_id,
                    notes=""
                ),
            )
            return empty_list
        return self.foods[group_id]
    
    def food_pies(self, group_id):
        all_food = self.all_foods(group_id)
        food_pies = list()
        sum_value = 0
        for food in all_food:
            value = food.money_cost * food.quantity_left_percentage / 100
            sum_value += value
        for food in all_food:
            if sum_value == 0:
                food_pies.append(
                    FoodPie(
                        name=food.name,
                        rate=1
                    ),
                )
                break
            else:
                food_pies.append(
                    FoodPie(
                        name=food.name,
                        rate=(food.money_cost * food.quantity_left_percentage / 100) / sum_value
                    ),
                )
        return food_pies
    
    def use_food(self, record: FoodUsageRecord):
        food = self.single_food(address=record.use_food_address, group_id=record.use_food_group_id)
        if food is None:
            return {
                "message": "食材不存在"
            }
        price = food.money_cost
        if record.use_food_group_id not in self.groups:
            return {
                "message": "食材不存在目前的 group"
            }
        for i in range(len(self.groups[record.use_food_group_id])):
            if self.groups[record.use_food_group_id][i].address == record.user_address:
                self.groups[record.use_food_group_id][i].balance -= (record.use_food_amount_percentage / 100 * price)
                for j in range(len(self.foods[record.use_food_group_id])):
                    if self.foods[record.use_food_group_id][j].food_address == record.use_food_address:
                        self.foods[record.use_food_group_id][j].quantity_left_percentage -= record.use_food_amount_percentage
                        break
                break
        return {
            "message": "成功使用食材"
        }
        
    def single_account(self, address: str, group_id: str):
        if group_id not in self.groups:
            return None
        for ac in self.groups[group_id]:
            if ac.address == address:
                return ac
        return None

    def all_accounts(self, group_id):
        if group_id not in self.groups:
            return None
        return self.groups[group_id]
    
    def account_join_group(self, aj: AccountJoin):
        already_in = False

        if aj.group_id not in self.groups:
            self.groups[aj.group_id] = list()

        for i in range(len(self.groups[aj.group_id])):
            if self.groups[aj.group_id][i].address == aj.address:
                already_in = True
                self.groups[aj.group_id][i].name = aj.name
                break
        # for ac in self.groups[aj.group_id]:
        #     if ac.address == aj.address:
        #         already_in = True
        #         break

        if not already_in:
            self.groups[aj.group_id].append(
                Account(
                    name=aj.name,
                    address=aj.address,
                    group_id=aj.group_id,
                    balance=0
                )
            )
            return {
                "message": "成功新增至群組"
            }
        else:
            return {
                "message": "已在群組中，名稱已修改"
            }

# class TransferRecord(BaseModel):
#     transfer_time: str
#     from_address: str
#     from_name: str
#     to_address: str
#     to_name: str
#     amount: float
#     group_id: str
#     notes: str | None = None
        
    def all_transactions(self, group_id: str):
        if group_id not in self.transactions:
            empty_list = list()
            empty_list.append(
                TransferRecord(
                    transfer_time="",
                    from_address="",
                    from_name="還款者",
                    to_address="",
                    to_name="收款者",
                    amount=0,
                    group_id=group_id,
                    notes=""
                ),
            )
            return empty_list
        return self.transactions[group_id]
    
    def transfer(self, record: TransferRequest):
        success_from = False
        success_to = False
        
        if record.group_id not in self.groups:
            return False

        from_name = ""
        to_name = ""
        for i in range(len(self.groups[record.group_id])):
            if self.groups[record.group_id][i].address == record.from_address:
                self.groups[record.group_id][i].balance += record.amount
                from_name = self.groups[record.group_id][i].name
                success_from = True
            elif self.groups[record.group_id][i].address == record.to_address: 
                self.groups[record.group_id][i].balance -= record.amount
                to_name = self.groups[record.group_id][i].name
                success_to = True
            if success_from and success_to:
                if record.group_id not in self.transactions:
                    self.transactions[record.group_id] = list()
                self.transactions[record.group_id].append(
                    TransferRecord(
                        transfer_time=record.transfer_time,
                        from_address=record.from_address,
                        from_name=from_name,
                        to_address=record.to_address,
                        to_name=to_name,
                        amount=record.amount,
                        group_id=record.group_id,
                        notes=record.notes
                    ),
                )
                break
        return success_from and success_to

In [13]:
db = Database()

# FastAPI

In [14]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware


In [15]:
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [16]:
@app.get("/")
async def read_root():
    return {"message": "查看 /docs 以獲得詳盡的 API Document"}

In [17]:
@app.get("/helloworld")
async def hello_world():
    """回傳 \"Hello, World!\""""
    return {
        "message": "Hello, World!"
    }

In [18]:
@app.get("/account/single")
async def single_account(address: str, group_id):
    """取得指定 account 的資料"""
    return {
        "message": f"地址為 {address} 的帳戶的資料",
        "data": db.single_account(address=address, group_id=group_id)
    }

In [19]:
@app.get("/account/all/")
async def all_account_data(group_id: str):
    """回傳指定 group 中所有 account 的資料"""
    return {
        "message": f"group_id: {group_id} 的所有 account 的資料",
        "data": db.all_accounts(group_id=group_id)
    }

In [20]:
@app.get("/account/transfer/")
async def all_transfer_records_in_group(group_id: str):
    """回傳指定 group 中所有的交易記錄"""
    return {
        "message": f"group_id: {group_id} 的所有交易記錄",
        "data": db.all_transactions(group_id=group_id)
    }

In [21]:
@app.post("/account/transfer/")
async def account_transfer(record: TransferRequest):
    if db.transfer(record):
        return {
            "message": "轉帳成功"
        }
    else:
        return {
            "message": "轉帳失敗"
        }

In [22]:
@app.post("/account/join_group/")
async def join_group(aj: AccountJoin):
    print()
    print()
    print(aj)
    print()
    print()    
    """將帳戶加入群組"""
    return db.account_join_group(aj=aj)

In [23]:
@app.get("/food/single/")
async def single_food_data(address: str, group_id: str):
    """取得指定 address 食物的資料"""
    return db.single_food(address=address, group_id=group_id)

In [24]:
@app.get("/food/all/")
async def get_all_food(group_id: str):
    """取得指定 group 中所有的食物資料"""
    return db.all_foods(group_id=group_id)

In [25]:
@app.get("/food/pie/")
async def get_food_pie(group_id: str):
    """取得指定 group 中所有的食物的百分比"""
    return db.food_pies(group_id=group_id)


In [26]:
@app.post("/food/insert/")
async def insert_food(fi: FoodInsert):
    """新增食物"""
    return db.insert_food(fi)

In [27]:
@app.post("/food/use/")
async def use_food(record: FoodUsageRecord):
    """使用指定的食物"""
    return db.use_food(record=record)

In [28]:
import uvicorn
config = uvicorn.Config(app, host="0.0.0.0", port=25565)
server = uvicorn.Server(config)
await server.serve()

INFO:     Started server process [18000]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:25565 (Press CTRL+C to quit)


INFO:     101.136.169.244:2241 - "GET /food/pie/?group_id=123 HTTP/1.1" 200 OK
INFO:     101.136.169.244:2322 - "GET /food/all/?group_id=123 HTTP/1.1" 200 OK
INFO:     101.136.169.244:2324 - "GET /account/single/?address=0x2766896449cad4afae653d7fd27201f526d5d4e8&group_id=123 HTTP/1.1" 307 Temporary Redirect
INFO:     101.136.169.244:2341 - "GET /account/all/?group_id=123 HTTP/1.1" 200 OK
INFO:     101.136.169.244:2342 - "GET /account/transfer/?group_id=123 HTTP/1.1" 200 OK
INFO:     101.136.169.244:2324 - "GET /account/single?address=0x2766896449cad4afae653d7fd27201f526d5d4e8&group_id=123 HTTP/1.1" 200 OK
INFO:     101.136.169.244:2982 - "GET /account/all/?group_id=123 HTTP/1.1" 200 OK
INFO:     101.136.169.244:2983 - "GET /account/transfer/?group_id=123 HTTP/1.1" 200 OK
INFO:     101.136.169.244:2972 - "GET /account/single/?address=0x2766896449cad4afae653d7fd27201f526d5d4e8&group_id=123 HTTP/1.1" 307 Temporary Redirect
INFO:     101.136.169.244:2972 - "GET /account/single?address=0x2

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18000]
